In [78]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [79]:
def synthetic_data(w, b, num_examples):
    """生成y = xw + b +噪声"""
    x = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(x, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return x, y.reshape((-1, 1))

In [80]:
true_w = torch.tensor([2 ,-3.4],dtype=torch.float)
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [81]:
print("features", features[0],'\nlabels', labels[0])

In [82]:
d2l.set_figsize()
d2l.plt.scatter(features[:, 1].detach().numpy(), labels.detach().numpy(),1 )

In [83]:
# 生成迭代器
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = torch.randperm(num_examples)
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [84]:
batch_size = 10

In [85]:
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

In [86]:
w = torch.normal(0,0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [87]:
# 线性回归模型
def linear_regression(X, w, b):
    return torch.matmul(X, w) + b

In [88]:
# 均方损失
def square_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [89]:
# 小批量随机梯度下降
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [90]:
lr = 0.03
num_epochs = 5
net = linear_regression
loss = square_loss

In [91]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch}, loss: {float(train_l.mean()):f}')

In [92]:
w

In [93]:
b

# 线性回归的简洁实现

In [94]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [95]:
true_w = torch.tensor([2 ,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [96]:
def load_array(data_array, batch_size, is_train=True):
    """构造一个迭代器"""
    dataset = data.TensorDataset(*data_array)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [97]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

In [98]:
from  torch import nn
net = nn.Sequential(nn.Linear(2, 1))

In [99]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

In [100]:
loss = nn.MSELoss()

In [101]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [102]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch}, loss: {l:f}')